# Analysis 0: Preprocessing
- Drop missing variables with > 60% missingness
- Demographics table 

## Import libraries

In [2]:
%load_ext rpy2.ipython 
# Load the R magic extension

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import patsy
from lifelines import CoxPHFitter 
import statsmodels as sm
from pathlib import Path

In [4]:
# Add the directory to sys.path
import sys
module_path = Path('./../code')
sys.path.append(str(module_path))
import utils

In [5]:
# Import libraries to allow data to be passed between Python and R env
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects import r

pandas2ri.activate()

In [6]:
%%R
# Run this cell to install packages the first time. 

# install.packages("bshazard")
# install.packages("survival")
# install.packages("tidyr")
# install.packages("dplyr")
# install.packages("magrittr")
# install.packages("tableone")
# install.packages("pROC")
# install.packages("PRROC")
# install.packages("caret")
# install.packages("survivalROC")
# install.packages("survminer")
# install.packages("scales")
# install.packages("broom")
# install.packages("purrr")


NULL


In [7]:
%%R
library(bshazard)
library(survival)
library(tidyr)
library(dplyr)
library(magrittr)
library(tableone)
library(pROC)
library(PRROC)
library(caret)
library(survivalROC)
library(survminer)
library(scales)
library(finalfit)
library(broom)
library(broom.helpers)
library(purrr)

R[write to console]: Loading required package: splines

R[write to console]: Loading required package: survival

R[write to console]: Loading required package: Epi

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘magrittr’


R[write to console]: The following object is masked from ‘package:tidyr’:

    extract


R[write to console]: Type 'citation("pROC")' for a citation.

R[write to console]: 
Attaching package: ‘pROC’


R[write to console]: The following objects are masked from ‘package:stats’:

    cov, smooth, var


R[write to console]: Loading required package: rlang

R[write to console]: 
Attaching package: ‘rlang’


R[write to console]: The following object is masked from ‘package:magrittr’:

    set_names


R[write to consol

In [8]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

# Displays all the columns, does 
pd.set_option('display.max_columns', None)

## Import data

In [9]:
demographics_table_filename = './../results/demographics_table.csv'
univariate_filename = './../results/univariate_analysis.csv'
multivariate_filename = './../results/multivariate_analysis.csv'

In [10]:
data_filename = './../data/cleaned_cohort_20250424.csv'
df = pd.read_csv(data_filename)[utils.VARS_TO_ANALYZE]
df.shape

(1745288, 79)

In [11]:
df.ugica.value_counts()

ugica
0.0    1744975
1.0        313
Name: count, dtype: int64

In [12]:
# Pass the DataFrame into the R environment
def pass_df(df, r_df_name):
    ro.globalenv[r_df_name] = df

ro.globalenv['numerical_vars'] = utils.NUMERICAL_VARS
ro.globalenv['categorical_vars'] = utils.CATEGORICAL_VARS # + ['sex_clean']
ro.globalenv['demographics_table_filename'] = str(demographics_table_filename)
ro.globalenv['univariate_filename'] = str(univariate_filename)
ro.globalenv['multivariate_filename'] = str(multivariate_filename)

##### Add variables to compare with current guidelines for risk-factor triggered screening for EAC

In [13]:
def num_risk_factors(row):
    score = 0 

    if row.age > 50: 
        score += 1 
    if row.sex == 'MALE':
        score += 1 
    if row.race_clean == 'White':
        score += 1
    if row.tobacco_binary == '1':
        score += 1
    if row.gerd == '1':
        score += 1
    if row.BMI_baseline >= 30:
        score += 1 
    if row.famhx_esophagealca or row.famhx_barretts:
        score += 1 
    
    return score 

df['eac_risk_factors_screening'] = df.apply(lambda x: num_risk_factors(x), axis=1)
df['meets_screening'] = (df.eac_risk_factors_screening >= 3).astype(int)

In [14]:
pass_df(df, "r_df")

/Users/alyssa.chen/miniconda3/envs/rpy2_env/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "subtype". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '


## Demographics table

In [15]:
%%R 
vars_to_analyze <- unlist(c(categorical_vars, numerical_vars))

demtable <- CreateTableOne(
    vars = vars_to_analyze,
    data = r_df,
    factorVars = unlist(categorical_vars),
    strata = "ugica",
    addOverall = TRUE,
    includeNA = TRUE
)
demtable_df <- print(demtable, quote = FALSE, noSpaces = TRUE, printToggle = FALSE, missing = TRUE)

write.csv(demtable_df, file = demographics_table_filename)

## Preprocessing

In [16]:
%%R 
# Ignore these columns
cols_to_ignore <- c(
    'months_to_event', 'ugica', 'death', 'subtype', 'visit_year', 'diagnosis_year', 'encounter_type', 'social_language', 
    'days_to_event', 'days_to_dx', 'days_to_death',
    "eac_risk_factors_screening", "meets_screening"
)

### Remove variables that have >60% missing

In [17]:
%%R 
missing_vars <- names(which(sapply(r_df, function(x) mean(is.na(x))) > 0.60))
missing_vars <- missing_vars[!missing_vars %in% cols_to_ignore]
missing_vars

 [1] "ugica_ESCC"                     "ugica_EAC"                     
 [3] "ugica_CGC"                      "ugica_NCGC"                    
 [5] "alcohol_all_missing"            "alcohol_binary_missing"        
 [7] "hpylori_active_missing"         "hpylori_active_chronic_missing"
 [9] "hgball_baseline"                "hgb_baseline"                  
[11] "mcv_baseline"                   "wbc_baseline"                  
[13] "plt_baseline"                   "sodium_baseline"               
[15] "potassium_baseline"             "chloride_baseline"             
[17] "bicarbonate_baseline"           "bun_baseline"                  
[19] "scr_baseline"                   "magnesium_baseline"            
[21] "calcium_baseline"               "phosphate_baseline"            
[23] "ast_baseline"                   "alt_baseline"                  
[25] "alp_baseline"                   "tbili_baseline"                
[27] "tprotein_baseline"              "albumin_baseline"              
[29] "

In [18]:
%%R 
print(dim(r_df))
r_df_nonmissing <- r_df[, !names(r_df) %in% missing_vars]
print(dim(r_df_nonmissing))

[1] 1745288      81
[1] 1745288      48


### Normalize continuous variables

In [19]:
%%R 
vars_to_normalize <- names(r_df_nonmissing)[
    !names(r_df_nonmissing) %in% cols_to_ignore &
    names(r_df_nonmissing) %in% numerical_vars
]

preproc <- preProcess(r_df_nonmissing[, vars_to_normalize], method = c("center", "scale"))
r_df_normal <- predict(preproc, r_df_nonmissing)
dim(r_df_normal)


[1] 1745288      48


### Factorize categorical variables

In [20]:
%%R
# Remove categorical variables that we do not want to do univariate analysis on
vars_categorical_for_univariate <- names(r_df_normal)[
    !names(r_df_normal) %in% cols_to_ignore &
    names(r_df_normal) %in% categorical_vars
]

# Factorize 
for (i in vars_categorical_for_univariate) {
    r_df_normal[[i]] <- factor(r_df_normal[[i]])
}

# Set reference levels 
r_df_normal$race_clean <- relevel(r_df_normal$race_clean, ref = 'White')
r_df_normal$race_clean_missing <- relevel(r_df_normal$race_clean_missing, ref = 'White')
r_df_normal$ethnicity <- relevel(r_df_normal$ethnicity, ref='Not Hispanic or Latino')
r_df_normal$ethnicity_missing <- relevel(r_df_normal$ethnicity_missing, ref='Not Hispanic or Latino')
r_df_normal$alcohol_all <- relevel(r_df_normal$alcohol_all, ref = '0.0')
# r_df_normal$alcohol_all_missing <- relevel(r_df_normal$alcohol_all_missing, ref = '0')
r_df_normal$alcohol_binary <- relevel(r_df_normal$alcohol_binary, ref = '0')
# r_df_normal$alcohol_binary_missing <- relevel(r_df_normal$alcohol_binary_missing, ref = '0')
r_df_normal$tobacco_all <- relevel(r_df_normal$tobacco_all, ref = '0.0')
r_df_normal$tobacco_all_missing <- relevel(r_df_normal$tobacco_all_missing, ref = '0')
r_df_normal$tobacco_binary <- relevel(r_df_normal$tobacco_binary, ref = '0')
r_df_normal$tobacco_binary_missing <- relevel(r_df_normal$tobacco_binary_missing, ref = '0')
r_df_normal$hpylori_active <- relevel(r_df_normal$hpylori_active, ref = '0.0')
# r_df_normal$hpylori_active_missing <- relevel(r_df_normal$hpylori_active_missing, ref = '0')
r_df_normal$hpylori_active_chronic <- relevel(r_df_normal$hpylori_active_chronic, ref = '0.0')
# r_df_normal$hpylori_active_chronic_missing <- relevel(r_df_normal$hpylori_active_chronic_missing, ref = '0')
r_df_normal$hpylori_active_chronic_binary <- relevel(r_df_normal$hpylori_active_chronic_binary, ref = '0')

In [26]:
%%R 
write.csv(r_df_normal, "./..data/df_analysis0.csv", row.names = FALSE)

R[write to console]: Error in file(file, ifelse(append, "a", "w")) : 
  cannot open the connection

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In min(x, na.rm = TRUE) :
R[write to console]: 
 
R[write to console]:  no non-missing arguments to min; returning Inf

R[write to console]: 2: 
R[write to console]: In max(x, na.rm = TRUE) :
R[write to console]: 
 
R[write to console]:  no non-missing arguments to max; returning -Inf

R[write to console]: 3: 
R[write to console]: In StdDiff(variable = var, group = strataVar) :
R[write to console]: 
 
R[write to console]:  Variable has only NA's in at least one stratum. na.rm turned off.

R[write to console]: 4: 
R[write to console]: In file(file, ifelse(append, "a", "w")) :
R[write to console]: 
 
R[write to console]:  cannot open file './..data/df_analysis0.csv': No such file or directory




Error in file(file, ifelse(append, "a", "w")) : 
  cannot open the connection


RInterpreterError: Failed to parse and evaluate line 'write.csv(r_df_normal, "./..data/df_analysis0.csv", row.names = FALSE)\n'.
R error message: 'Error in file(file, ifelse(append, "a", "w")) : \n  cannot open the connection'